In [1]:
# import shutil
import pandas as pd
import numpy as np
# import transformers
from transformers import AutoTokenizer
import torch
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
import ast
# import json
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

pretrained = 'distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained)

MAX_LEN = 400
# TRAIN_BATCH_SIZE = 32
# VALID_BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 1e-05
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [400]])


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MinMaxScaler(clip=True)

In [2]:
train_data = pd.read_parquet("steps400w1640k.gzip")

In [7]:
train_data

,samples,tag
0,"In a heavy 2-quart saucepan, mix brown sugar, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Place chipped beef on bottom of baking dish. P...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"In a slow cooker, combine all ingredients. Cov...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
3,Boil and debone chicken. Put bite size pieces ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Combine first 4 ingredients and press in 13 x ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1642942,"Mix tuna, egg, tomato, red onion and Hellmann'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1642943,"In a small sauce pan, melt butter and add flou...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
1642944,"For the cucumber salad, mix the vinegar and su...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1642945,2. Beat egg white with evaporated skim milk. P...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
new_traindata = train_data[train_data.tag.str.contains(3, regex=False) == True]
new_traindata = new_traindata.reset_index(drop=True)
# new_traindata['tag'] = new_traindata['tag'].apply(ast.literal_eval)


In [4]:
df10 = new_traindata.sample(10, random_state=123)

In [10]:
df10 = df10.drop(columns=['tag']).reset_index(drop=True)

In [13]:
df10.to_csv('../../data/reportdata2.csv')

In [9]:
new_traindata.sample(1).reset_index(drop=True).iloc[0][0]

'Combine sugar and wine in a large nonaluminum Dutch oven or stockpot; stir well. Cook over medium-high heat 2 minutes or until sugar dissolves, stirring constantly (do not boil). Add apricots, cinnamon stick, and vanilla bean; bring to a boil, and cook 30 seconds, stirring gently after 15 seconds. Remove from heat; gently stir in vodka. Let cool completely. Spoon mixture into a large bowl; cover loosely with cheesecloth or a towel, and refrigerate 4 days. Drain apricot mixture through a fine sieve, reserving liqueur and marinated apricots. Discard cinnamon stick and vanilla bean. Pour liqueur into decorative bottles, and store in the refrigerator. Spoon the marinated apricots into a bowl; cover and chill. Note: Apricot liqueur will keep in the refrigerator for several weeks. Use the marinated apricots within 3 days.'

In [10]:
print(len(train_data), len(new_traindata))

1642947 1191963


In [11]:
sents = []
targets = []
count = 0
for row in new_traindata.iterrows():
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    target = [0]*sent_len
    for i in range(sent_len):
        if row[1]['tag'][i] == 3:
            if sent[i][0] == '(':
                target[i] = float(sent[i][1:])
            elif 'time' in sent[i]:
                continue
            else:
                target[i] = float(sent[i])
            sent[i] = '[MASK]'
    sents.append(sent)
    targets.append(target)
    count += 1
    
times = []
for row in new_traindata.iterrows():
    temp_r = []
    temp_sub = []
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    target = [0]*sent_len
    for i in range(sent_len):
        if row[1]['tag'][i] == 3:
            if sent[i][0] == '(':
                temp_sub.append(float(sent[i][1:]))
            elif 'time' in sent[i]:
                continue
            else:
                temp_sub.append(float(sent[i]))
        elif row[1]['tag'][i] == 4:
            temp_sub.append(sent[i])
            temp_r.append(temp_sub)
            temp_sub = []
    times.append(temp_r)
minutized_target = []
for r in times:
    temp = 0
    for pair in r:
        if len(pair) == 2:
            if 'minute' in pair[1]:
                temp += pair[0]
            elif 'second' in pair[1]:
                temp += pair[0]/60
            elif 'hour' in pair[1]:
                temp += pair[0]*60
    minutized_target.append([temp])
# print(minutized_target)
# raise RuntimeError("")
joined_sents = [" ".join(s) for s in sents]
processed_training_df = pd.DataFrame()
processed_training_df['sentences'] = joined_sents
processed_training_df['target'] = targets
processed_training_df['minute_target'] = scaler.transform(minutized_target)
processed_training_df['tag'] = new_traindata['tag']

In [12]:
processed_training_df

,sentences,target,minute_target,tag
0,"In a heavy 2-quart saucepan, mix brown sugar, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.08750,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Place chipped beef on bottom of baking dish. P...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.45000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"In a slow cooker, combine all ingredients. Cov...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.0, 0...",0.60000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
3,Boil and debone chicken. Put bite size pieces ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.05000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Combine first 4 ingredients and press in 13 x ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.05000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
1191958,"In blender container, puree all ingredients. M...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.01250,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1191959,"In a small sauce pan, melt butter and add flou...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.0, 0...",0.00500,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, ..."
1191960,"For the cucumber salad, mix the vinegar and su...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1191961,2. Beat egg white with evaporated skim milk. P...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.10000,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
processed_training_df.to_parquet("1200k_processed_gathered_steps.gzip", compression="gzip")

In [5]:
new_traindata = pd.read_csv("steps400w100000t.csv")
new_traindata['tag'] = new_traindata['tag'].apply(ast.literal_eval)

sents = []
targets = []
count = 0
for row in new_traindata.iterrows():
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    target = [0]*sent_len
    for i in range(sent_len):
        if row[1]['tag'][i] == 3:
            if sent[i][0] == '(':
                target[i] = float(sent[i][1:])
            elif 'time' in sent[i]:
                continue
            else:
                target[i] = float(sent[i])
            sent[i] = '[MASK]'
    sents.append(sent)
    targets.append(target)
    count += 1
    
times = []
for row in new_traindata.iterrows():
    temp_r = []
    temp_sub = []
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    target = [0]*sent_len
    for i in range(sent_len):
        if row[1]['tag'][i] == 3:
            if sent[i][0] == '(':
                temp_sub.append(float(sent[i][1:]))
            elif 'time' in sent[i]:
                continue
            else:
                temp_sub.append(float(sent[i]))
        elif row[1]['tag'][i] == 4:
            temp_sub.append(sent[i])
            temp_r.append(temp_sub)
            temp_sub = []
    times.append(temp_r)
minutized_target = []
for r in times:
    temp = 0
    for pair in r:
        if len(pair) == 2:
            if 'minute' in pair[1]:
                temp += pair[0]
            elif 'second' in pair[1]:
                temp += pair[0]/60
            elif 'hour' in pair[1]:
                temp += pair[0]*60
    minutized_target.append([temp])
joined_sents = [" ".join(s) for s in sents]
processed_training_df = pd.DataFrame()
processed_training_df['sentences'] = joined_sents
processed_training_df['target'] = targets
processed_training_df['minute_target'] = scaler.transform(minutized_target)
processed_training_df['tag'] = new_traindata['tag']

In [8]:
out_df = processed_training_df[['sentences','minute_target']]

In [10]:
out_df.to_parquet('steps400w100000t.gzip',compression='gzip')

In [3]:
import random
new_traindata = pd.read_csv("steps400w100000t.csv")
# new_traindata = train_data[train_data['tag'].str.contains('3') == True]
# new_traindata = new_traindata.reset_index(drop=True)
new_traindata['tag'] = new_traindata['tag'].apply(ast.literal_eval)
# new_traindata = new_traindata.sample(frac=1)
traindata_t, traindata_f = np.split(new_traindata.sample(frac=1,random_state=333),
                                                      [int(.5*len(new_traindata))])
ttag = [[1,0]]*len(traindata_t)
ftag = [[0,1]]*len(traindata_f)
def randhalf():
    return random.randint(0,20)*0.5

sents = []
targets = []
count = 0
for row in traindata_f.iterrows():
    sent = row[1]['samples'].split()
    sent_len = len(row[1]['tag'])
    # target = [0]*sent_len
    bracket = False
    for i in range(sent_len):
        if row[1]['tag'][i] == 3:
            if sent[i][0] == '(':
                target = float(sent[i][1:])
                bracket = True
            elif 'time' in sent[i]:
                continue
            else:
                target = float(sent[i])
            sent[i] = '('+str(target*randhalf()) if bracket else str(target*randhalf())
        bracket = False
    sents.append(sent)
    # targets.append(target)
    count += 1

joined_sents = [" ".join(s) for s in sents]

fdf = pd.DataFrame()
fdf['sentences'] = joined_sents
fdf['target'] = ftag

tdf = pd.DataFrame()
tdf['sentences'] = traindata_t['samples']
tdf['target'] = ttag
processed_training_df = pd.concat([fdf,tdf]).reset_index(drop=True)

In [8]:
processed_training_df

,sentences,target
0,Rinse mushrooms and pat dry. Remove stems; cho...,"[0, 1]"
1,Chop artichoke hearts; mix with other ingredie...,"[0, 1]"
2,"Cream shortening; gradually add sugar, beating...","[0, 1]"
3,*Use water if fresh peaches.,"[0, 1]"
4,"Mix pretzels, 3 tablespoons sugar and margarin...","[0, 1]"
...,...,...
6533,Bake shell in very hot oven (450 degrees) for ...,"[1, 0]"
96434,Preheat oven to 350 degrees. Grease a 13 x 9-i...,"[1, 0]"
87966,"`Boil sugar, water and salt to hard boil stage...","[1, 0]"
29534,Brown chops slowly on both sides in oil. Remov...,"[1, 0]"


In [7]:
processed_training_df.to_parquet("10w_for_fsb.gzip", compression="gzip")

In [34]:
somedf = pd.read_parquet("steps400w100000t.gzip")

In [35]:
somedf

,sentences,minute_target
0,"In a heavy 2-quart saucepan, mix brown sugar, ...",0.0875
1,Place chipped beef on bottom of baking dish. P...,0.4500
2,"In a slow cooker, combine all ingredients. Cov...",0.6000
3,Boil and debone chicken. Put bite size pieces ...,0.0500
4,Combine first 4 ingredients and press in 13 x ...,0.0500
...,...,...
99995,Whisk the first 5 ingredients together in bowl...,0.0000
99996,"Beat shortening, sugar, pumpkin and egg in mix...",0.0000
99997,Stir all ingredients and chill.,0.0000
99998,"Wash and rinse beans carefully, selecting out ...",0.3000


In [18]:
import pandas as pd
df500 = pd.read_parquet("steps400w500k_t.gzip")

In [19]:
df500.iloc[0][0]

'Saute zucchini and chopped onion in oil. Whip cottage cheese with basil and oregano in blender. Place alternating layers of zucchini, cottage cheese and tomatoes in a 1.5-quart casserole dish. Top with Parmesan cheese. Bake at 350 degrees uncovered for <mask> minutes.'